# NVIDIA Data Visualization


In [1]:
# use alpaca as data api 
import os 
from typing import List
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process.kernels as K 
import matplotlib.pyplot as plt 

from matplotlib.pyplot import figure
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

import yaml
import redis
import json

def acquire_credentials(secret_path: str = None):
    """
    looks for the credentials in a secret file
    """
    if secret_path is None:
        secret_path = PATH_SECRETS
    else:
        pass

    if os.path.exists(secret_path):

        # open file to secrets
        with open(secret_path, "r") as f:

            # use yaml to safely load
            key_information = yaml.safe_load(f)
            return key_information["api_key"], key_information["api_secret"]

    else:
        logger.critical("Secrets file not found")
        raise FileNotFoundError(f"File not found for {secret_path}")


In [2]:
# download data functions 

def download_data(secrets_path, start_date):
    """
    downloads data from the alpaca 
    """
    api_key , api_secret = acquire_credentials(secrets_path)

    client = StockHistoricalDataClient(api_key=api_key,
                                       secret_key=api_secret)

    request_params = StockBarsRequest(
        symbol_or_symbols="NVDA",
        timeframe=TimeFrame.Day,
        start=datetime.strptime(start_date, '%Y-%m-%d')
    )

    bars = client.get_stock_bars(request_params)
    df = bars.df 
    timestamps = df.index.map(lambda x : x[1]).values
    closing_prices = df['close'].values

    interim_df = pd.DataFrame({'timestamp':timestamps, 
                               'price': closing_prices})
    
    return interim_df        

In [3]:
SECRETS_PATH = "../src/.secrets"
START_DATE = "2024-01-04"

In [4]:
df = download_data(SECRETS_PATH, START_DATE)

In [5]:
# save raw data to redis 
connection =  redis.Redis(host='red', port=6379)

In [6]:
# stock list 

stock_list = connection.get('_STOCKS')

if stock_list is None:
    connection.set('_STOCKS', json.dumps(['NVDA']))
else:
    stock_list = json.loads(stock_list)
    if 'NVDA' not in stock_list:
        stock_list.append('NVDA')
        connection.set('_STOCKS', json.dumps(stock_list))


b'["NVDA"]'